<a href="https://colab.research.google.com/github/joshIsac/LargeLanguageModel/blob/main/llm__lab6_seqtoseqtransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#importing library for Seq2seq transformer
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score

In [10]:
#import the datasets for language translation using datasets library
dataset = load_dataset("wmt14", "de-en",split='train[:1000]')
#split the dataset
train_dataset = dataset.train_test_split(test_size=0.2)
train_data = train_dataset['train']
val_data = train_dataset['test']


In [12]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

def tokenize_function(examples):
    inputs = [example['en'] for example in examples['translation']]
    targets = [example['de'] for example in examples['translation']]
    # Enable padding and truncation
    model_inputs = tokenizer(inputs, text_target=targets, truncation=True, padding="max_length")
    return model_inputs

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [15]:
# Apply the tokenizer to the datasets
tokenized_train_data = train_data.map(tokenize_function, batched=True)
tokenized_valid_data = val_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [16]:
from transformers import T5ForConditionalGeneration, T5Config

# Load the model configuration and the model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [17]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_valid_data,
    tokenizer=tokenizer,
)

# Start training
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.196412
2,No log,0.161643
3,1.583900,0.155513


TrainOutput(global_step=600, training_loss=1.3549631881713866, metrics={'train_runtime': 240.2781, 'train_samples_per_second': 9.988, 'train_steps_per_second': 2.497, 'total_flos': 324820323532800.0, 'train_loss': 1.3549631881713866, 'epoch': 3.0})

In [18]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation metrics
print(f"Evaluation Results: {eval_results}")


Evaluation Results: {'eval_loss': 0.1555132269859314, 'eval_runtime': 6.1306, 'eval_samples_per_second': 32.623, 'eval_steps_per_second': 8.156, 'epoch': 3.0}


In [23]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)
tokenizer = T5Tokenizer.from_pretrained("t5-small")

sample_text = "It is wonderful to play"
inputs = tokenizer(sample_text, return_tensors="pt").input_ids.to(device)

# Generate translation
outputs = model.generate(inputs, max_length=40, num_beams=4, early_stopping=True)
translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Translation: {translation}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Translation: Es ist wunderbar, spielen zu spielen.
